<h1>Segmenting and Clustering Neighborhoods in Bangalore
    

## Creating a dataframe

In [1]:
import pandas as pd
!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup

import requests
import numpy as np
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |   py36h9f0ad1d_0         160 KB  conda-forge
    bs4-4.9.0                  |                0           4 KB  conda-forge
    soupsieve-1.9.4            |   py36h9f0ad1d_1          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         222 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.0-py36h9f0ad1d_0
  bs4                conda-forge/noarch::bs4-4.9.0-0
  soupsieve          conda-forge/linux-64::soupsieve-1.9.4-py36h9f0ad1d_1



soupsieve-1.9.4      | 58 KB     | ##################################### | 100% 
beautifuls

In [2]:
source = requests.get("http://www.geonames.org/export/zip/").text
soup = BeautifulSoup(source, 'html.parser')
df = pd.read_csv('IN.txt',delimiter="\t",header = None)
df.drop(df.columns[[0, 4, 6, 8, 11]], axis = 1, inplace = True)
df.columns = ['Pin Code', 'Neighborhood','State','District','Borough','Latitude','Longitude']
df = df.dropna()
df.head()

,Pin Code,Neighborhood,State,District,Borough,Latitude,Longitude
0,744101,Marine Jetty,Andaman & Nicobar Islands,South Andaman,Portblair,11.6667,92.7500
1,744101,Port Blair,Andaman & Nicobar Islands,South Andaman,Port Blair,11.6667,92.7500
2,744101,N.S.Building,Andaman & Nicobar Islands,South Andaman,Portblair,11.6667,92.7500
3,744102,Haddo,Andaman & Nicobar Islands,South Andaman,Port Blair,11.6833,92.7167
4,744102,Chatham,Andaman & Nicobar Islands,South Andaman,Portblair,11.7000,92.6667


In [3]:
df_bangalore= df[df['Borough'].str.contains("Bangalore")].reset_index(drop=True)
df_bangalore

,Pin Code,Neighborhood,State,District,Borough,Latitude,Longitude
0,560001,Bangalore G.P.O.,Karnataka,Bangalore,Bangalore North,12.9914,77.5944
1,560001,Legislators Home,Karnataka,Bangalore,Bangalore North,12.9914,77.5944
2,560001,Mahatma Gandhi Road,Karnataka,Bangalore,Bangalore North,12.9914,77.5944
3,560001,Vidhana Soudha,Karnataka,Bangalore,Bangalore North,12.9914,77.5944
4,560001,Rajbhavan (Bangalore),Karnataka,Bangalore,Bangalore North,12.9914,77.5944
...,...,...,...,...,...,...,...
253,560110,Ullalu Upanagar,Karnataka,Bangalore,Bangalore,13.1791,77.0942
254,562130,Tavarekere (Bangalore),Karnataka,Bangalore,Bangaloresouth,13.9288,77.0614
255,562157,Bettahalsur,Karnataka,Bangalore,Bangalore North,13.1438,77.6169
256,562162,Madanayakanahalli,Karnataka,Bangalore,Bangalore North,17.8647,77.8229


## Explore and Cluster Neighborhoods in Bangalore

In [4]:
address = 'Bangalore, KA'

geolocator = Nominatim(user_agent="in_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


In [5]:
map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_bangalore['Latitude'], df_bangalore['Longitude'], df_bangalore['Borough'], df_bangalore['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore)  
    
map_bangalore

In [6]:
# @hidden_cell
CLIENT_ID = 'MWGCLOZUCHTUSXMGVT5TSGLM3Q13T4BYLSDVJZUYB2XS53CM'
CLIENT_SECRET = 'I3MPYYK4MEZTGNO20BPW1VJ2DOFARWO302VEJW4QV2OLD3YE' 
VERSION = '20180605'

In [7]:
#Getting the neighbourhood name,latitude and longitude
neighborhood_latitude = df_bangalore.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_bangalore.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_bangalore.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bangalore G.P.O. are 12.9914, 77.5944.


In [8]:
# Now to get 100 venues within Beaches within radius of 500m
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()


In [9]:
#Funtion to extract category
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
#cleaning
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Ujwal Bar & Restaurant,Indian Restaurant,12.992280,77.594473
1,Millers 46,Steakhouse,12.991666,77.594207
2,Alliance Française,Concert Hall,12.991232,77.596723
3,Infinitea,Tea Room,12.987157,77.594835
4,Imperial Restaurant,Indian Restaurant,12.991150,77.593837


In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

34 venues were returned by Foursquare.


In [12]:
#Exploring neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
bangalore_venues = getNearbyVenues(names=df_bangalore['Neighborhood'],
                                   latitudes=df_bangalore['Latitude'],
                                   longitudes=df_bangalore['Longitude']
                                  )
bangalore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bangalore G.P.O.,12.9914,77.5944,Ujwal Bar & Restaurant,12.992280,77.594473,Indian Restaurant
1,Bangalore G.P.O.,12.9914,77.5944,Millers 46,12.991666,77.594207,Steakhouse
2,Bangalore G.P.O.,12.9914,77.5944,Alliance Française,12.991232,77.596723,Concert Hall
3,Bangalore G.P.O.,12.9914,77.5944,Infinitea,12.987157,77.594835,Tea Room
4,Bangalore G.P.O.,12.9914,77.5944,Imperial Restaurant,12.991150,77.593837,Indian Restaurant


In [14]:
bangalore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A F Station Yelahanka,1,1,1,1,1,1
Achitnagar,2,2,2,2,2,2
Adugodi,4,4,4,4,4,4
Agara,41,41,41,41,41,41
Agram,1,1,1,1,1,1
...,...,...,...,...,...,...
Yelachenahalli,2,2,2,2,2,2
Yelahanka,3,3,3,3,3,3
Yemalur,56,56,56,56,56,56


## Analysing each Neighborhood

In [15]:
# one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangalore_onehot.columns[-1]] + list(bangalore_onehot.columns[:-1])
toronto_denc_onehot = bangalore_onehot[fixed_columns]

bangalore_onehot.head()

,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,...,Toll Plaza,Tourist Information Center,Track Stadium,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store,Yoga Studio,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
bangalore_grouped = bangalore_onehot.groupby('Neighborhood').mean().reset_index()
bangalore_grouped.head()

,Neighborhood,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Toll Plaza,Tourist Information Center,Track Stadium,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store,Yoga Studio,Zoo Exhibit
0,A F Station Yelahanka,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0
1,Achitnagar,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0
2,Adugodi,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0
3,Agara,0.0,0.02439,0.02439,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.02439,0.0,0.0,0.02439,0.0
4,Agram,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0


In [34]:
#Top 10 most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangalore_grouped['Neighborhood']

for ind in np.arange(bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A F Station Yelahanka,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
1,Achitnagar,Gym / Fitness Center,Health & Beauty Service,Zoo Exhibit,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
2,Adugodi,Indian Restaurant,Fast Food Restaurant,Design Studio,Bus Station,Zoo Exhibit,Eastern European Restaurant,Financial or Legal Service,Farmers Market,Event Service,Electronics Store
3,Agara,Café,Indian Restaurant,Coffee Shop,Pub,Ice Cream Shop,Lounge,Pizza Place,Bar,Paper / Office Supplies Store,Mexican Restaurant
4,Agram,Home Service,Zoo Exhibit,Donut Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Dive Bar


## K means Clustering

In [35]:
kclusters = 5

toronto_denc_grouped_clustering = bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_denc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 4, 4, 1, 1, 4, 2, 4, 4], dtype=int32)

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangalore_merged = df_bangalore

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bangalore_merged = bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


bangalore_merged.head() # check the last columns!



,Pin Code,Neighborhood,State,District,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,560001,Bangalore G.P.O.,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4.0,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
1,560001,Legislators Home,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4.0,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
2,560001,Mahatma Gandhi Road,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4.0,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
3,560001,Vidhana Soudha,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4.0,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
4,560001,Rajbhavan (Bangalore),Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4.0,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub


In [45]:
bangalore_merged.isnull()
bangalore_merged = bangalore_merged.dropna(how='any', subset=['Cluster Labels'])
bangalore_merged['Cluster Labels'] =bangalore_merged['Cluster Labels'].astype(int)
bangalore_merged.head()


,Pin Code,Neighborhood,State,District,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,560001,Bangalore G.P.O.,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
1,560001,Legislators Home,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
2,560001,Mahatma Gandhi Road,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
3,560001,Vidhana Soudha,Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
4,560001,Rajbhavan (Bangalore),Karnataka,Bangalore,Bangalore North,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub


In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

bangalore_merged['Cluster Labels'] =bangalore_merged['Cluster Labels'].astype(int)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        bangalore_merged['Latitude'], 
        bangalore_merged['Longitude'], 
        bangalore_merged['Neighborhood'], 
        bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examing the Clusters

In [51]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 0, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
130,Subramanyapura,12.769,77.7432,0,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
131,Chikkalasandra,12.769,77.7432,0,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
132,Doddakallasandra,12.769,77.7432,0,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
133,Konanakunte,12.769,77.7432,0,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
134,Tataguni,12.769,77.7432,0,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
135,A F Station Yelahanka,12.769,77.7432,0,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
136,BSF Campus Yelahanka,12.769,77.7432,0,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
143,G.K.V.K.,12.769,77.7432,0,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
148,Devanagundi,12.769,77.7432,0,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
149,Kadugodi,12.769,77.7432,0,IT Services,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop


In [49]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 1, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Yeshwanthpur Bazar,13.11,77.4484,1.0,Tourist Information Center,Zoo Exhibit,Diner,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
50,Yeswanthpura,13.11,77.4484,1.0,Tourist Information Center,Zoo Exhibit,Diner,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
52,Anandnagar (Bangalore),13.11,77.4484,1.0,Tourist Information Center,Zoo Exhibit,Diner,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
53,Hebbal Kempapura,13.11,77.4484,1.0,Tourist Information Center,Zoo Exhibit,Diner,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
54,H.A. Farm,13.11,77.4484,1.0,Tourist Information Center,Zoo Exhibit,Diner,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
58,Deepanjalinagar,13.11,77.4484,1.0,Tourist Information Center,Zoo Exhibit,Diner,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop
59,Governmemnt Electric Factory,13.11,77.4484,1.0,Tourist Information Center,Zoo Exhibit,Diner,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop,Dive Bar,Dessert Shop


In [52]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 3, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
191,Kagalipura,12.7717,77.5053,3,Zoo Exhibit,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
192,Somanhalli,12.7717,77.5053,3,Zoo Exhibit,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
193,Udaypura,12.7717,77.5053,3,Zoo Exhibit,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
194,Thattekuppe,12.7717,77.5053,3,Zoo Exhibit,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
195,Taralu,12.7717,77.5053,3,Zoo Exhibit,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
196,Bolare,12.7717,77.5053,3,Zoo Exhibit,Creperie,Food & Drink Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop


In [54]:
bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 4, bangalore_merged.columns[[1] + list(range(5, bangalore_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bangalore G.P.O.,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
1,Legislators Home,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
2,Mahatma Gandhi Road,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
3,Vidhana Soudha,12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
4,Rajbhavan (Bangalore),12.9914,77.5944,4,Indian Restaurant,Coffee Shop,Italian Restaurant,Chinese Restaurant,Tea Room,Furniture / Home Store,Mediterranean Restaurant,Sandwich Place,Fast Food Restaurant,Pub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,Hampinagar,13.1791,77.0942,4,Gym / Fitness Center,Health & Beauty Service,Zoo Exhibit,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
250,Achitnagar,13.1791,77.0942,4,Gym / Fitness Center,Health & Beauty Service,Zoo Exhibit,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
252,Thalaghattapura,13.1791,77.0942,4,Gym / Fitness Center,Health & Beauty Service,Zoo Exhibit,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
253,Ullalu Upanagar,13.1791,77.0942,4,Gym / Fitness Center,Health & Beauty Service,Zoo Exhibit,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Event Service,Electronics Store,Eastern European Restaurant,Donut Shop
